In [1]:
from generisanje_trening_skupa import * 

ModuleNotFoundError: No module named 'utils'

In [2]:
from tqdm import tqdm

Ide se po rangovima i sa odredjenom verovatnocom za taj rang se bira koji ce fragmentni jon biti proizveden => zatim se proizvodi taj jon.
Za jedan peptid mozemo odraditi vise prolazaka, a ne samo jedan jer u realnosti biva submittovano X peptida, ne samo jedan

In [3]:
izabrani_joni, trening_skup = generisi_trening_skup(
    broj_peptida=500
)

In [4]:
for peptid, spektar in trening_skup.peptidi_i_spektri:
    spektar.mase_intenziteti.to_csv(f"izracunavanje-parametara-skoringa/trening-skup/{peptid}.csv")

In [5]:
broj_odgovarajucih = {}
objasnjeni_pikovi_ranga = {}
neobjasnjeni_pikovi_ranga = {}
broj_pikova_po_rangu = {}

for i in tqdm(range(len(trening_skup.peptidi_i_spektri))):
    peptid = trening_skup.peptidi_i_spektri[i][0].peptid
    spektar = dodeli_rangove_spektru_generating(trening_skup.peptidi_i_spektri[i][1].mase_intenziteti)

    duzina_peptida = len(peptid)
    broj_rangova = max(spektar.index.get_level_values(level="rang")) + 1

    for rang in range(1, broj_rangova):
        broj_pikova_po_rangu[rang] = broj_pikova_po_rangu.get(rang, 0) +  1
    
    for j in range(len(peptid)):
        prefiksna_masa = Peptid(peptid[:j]).masa
        sufiksna_masa = Peptid(peptid[j:]).masa 

        for rang in range(1, broj_rangova):

            if rang not in spektar.index.get_level_values(level="rang"):
                continue

            if rang not in broj_odgovarajucih:
                broj_odgovarajucih[rang] = {}
                
            for jon, pomeraj in celobrojni_pomeraj_fragmentnih_jona.items():

                if jon not in broj_odgovarajucih[rang]:
                    broj_odgovarajucih[rang][jon] = 0
                    
                # sufiksni peptidi
                if np.any([jon.startswith(oznaka) for oznaka in ["x","y","z"]]):
                    sufiksi_u_spektru = spektar.loc[[rang]][lambda df: 
                        (df.m_z >= sufiksna_masa + pomeraj - 0.5) & 
                        (df.m_z <= sufiksna_masa + pomeraj + 0.5)
                   ]

                    if len(sufiksi_u_spektru):
                        broj_odgovarajucih[rang][jon] += len(sufiksi_u_spektru)
                    #objasnjeni_pikovi_ranga[rang] = objasnjeni_pikovi_ranga.get(rang, []) + \
                        #list(sufiksi_u_spektru.m_z.values)

                # prefiksni peptidi
                elif np.any(jon.startswith(oznaka) for oznaka in ["a","b","c"]):
                    prefiksi_u_spektru = spektar.loc[[rang]][lambda df: 
                        (df.m_z >= prefiksna_masa + pomeraj - 0.5) &
                        (df.m_z <= prefiksna_masa + pomeraj + 0.5)
                    ]

                    if len(prefiksi_u_spektru):
                        broj_odgovarajucih[rang][jon] += len(prefiksi_u_spektru)  
                    #objasnjeni_pikovi_ranga[rang] = objasnjeni_pikovi_ranga.get(rang, []) + \
                        #list(prefiksi_u_spektru.m_z.values)


        #for rang in range(1, min(max(spektar.index)+1,42)):
        #    neobjasnjeni_pikovi_ranga[rang] = neobjasnjeni_pikovi_ranga.get(rang, 0) + len(list(set(spektar.loc[[rang]]["m_z"].values).difference(set(objasnjeni_pikovi_ranga[rang]))))

100%|█████████████████████████████████████| 500/500 [50:26<00:00,  6.05s/it]


In [6]:
eksperimentalna_verovatnoca_jona_po_rangovima = pd.DataFrame({
    rang: {
        jon: round(broj_odgovarajucih[rang][jon]/broj_pikova_po_rangu[rang],5) \
            if broj_pikova_po_rangu[rang] > 0 else 0
        for jon in po_jonima
    } for rang, po_jonima in broj_odgovarajucih.items()
})

In [16]:
broj_rangova = max(eksperimentalna_verovatnoca_jona_po_rangovima.columns)

In [8]:
eksperimentalna_verovatnoca_jona_po_rangovima.loc["noise"] = [round(max(0, 1 - eksperimentalna_verovatnoca_jona_po_rangovima[rang].sum()), 5) for rang in range(1, broj_rangova+1)]

In [9]:
eksperimentalna_verovatnoca_jona_po_rangovima.to_csv("izracunavanje-parametara-skoringa/parametri/verovatnoca_jona_po_rangovima_generating.csv")

In [17]:
eksperimentalna_verovatnoca_jona_po_rangovima

,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
y,0.762,0.764,0.828,0.782,0.770,0.774,0.794,0.764,0.740,0.488,...,0.034,0.032,0.04435,0.02664,0.02386,0.04051,0.02817,0.03472,0.00000,0.00
b,0.124,0.116,0.082,0.082,0.110,0.092,0.092,0.096,0.136,0.270,...,0.080,0.068,0.05040,0.06557,0.06941,0.05823,0.05634,0.06250,0.00000,0.00
b-h2o,0.046,0.052,0.042,0.052,0.058,0.070,0.042,0.038,0.052,0.054,...,0.108,0.100,0.08871,0.09836,0.12148,0.10127,0.09507,0.07639,0.14706,0.00
a,0.002,0.002,0.006,0.004,0.000,0.012,0.004,0.004,0.006,0.002,...,0.034,0.016,0.04234,0.03484,0.02386,0.01772,0.02465,0.02778,0.02941,0.00
b-nh3,0.014,0.010,0.010,0.006,0.006,0.006,0.012,0.008,0.004,0.006,...,0.056,0.064,0.05242,0.04098,0.05206,0.06329,0.05282,0.04861,0.05882,0.25
y-nh3,0.002,0.006,0.000,0.000,0.000,0.000,0.000,0.002,0.008,0.046,...,0.146,0.130,0.13508,0.16393,0.17787,0.12405,0.18310,0.12500,0.23529,0.00
b-h2o-h2o,0.004,0.006,0.004,0.004,0.004,0.010,0.006,0.004,0.010,0.002,...,0.108,0.084,0.10081,0.10451,0.09544,0.10886,0.13028,0.13889,0.02941,0.00
noise,0.046,0.044,0.028,0.070,0.052,0.036,0.050,0.084,0.044,0.132,...,0.434,0.506,0.48589,0.46517,0.43602,0.48607,0.42957,0.48611,0.50001,0.75


In [18]:
math.log(150)

5.0106352940962555

In [10]:
missing_ion_probs = {}
missing_ion_scores = {}
broj_mogucih = {jon:0 for jon,_ in celobrojni_pomeraj_fragmentnih_jona.items()}

for i in tqdm(range(len(trening_skup.peptidi_i_spektri))):
    peptid = trening_skup.peptidi_i_spektri[i][0].peptid
    spektar = dodeli_rangove_spektru_generating(trening_skup.peptidi_i_spektri[i][1].mase_intenziteti)
    
    for j in range(len(peptid)):
        prefiksna_masa = Peptid(peptid[:j]).masa
        sufiksna_masa = Peptid(peptid[j:]).masa 
        
        for jon, pomeraj in celobrojni_pomeraj_fragmentnih_jona.items(): 
            
            # sufiksni peptidi
            if np.any([jon.startswith(oznaka) for oznaka in ["x","y","z"]]):
                sufiksi_u_spektru = spektar[lambda df: 
                    (df.m_z >= sufiksna_masa + pomeraj - 0.5) & 
                    (df.m_z <= sufiksna_masa + pomeraj + 0.5)
               ]

                broj_mogucih[jon] += 1 if (((sufiksna_masa + pomeraj) >= 91) and ((sufiksna_masa + pomeraj) <= 2000)) else 0
                if not len(sufiksi_u_spektru):
                    missing_ion_probs[jon] = missing_ion_probs.get(jon, 0) + (1 if ((sufiksna_masa + pomeraj) >= 91) and ((sufiksna_masa + pomeraj) <= 2000) else 0)
                                
            # prefiksni peptidi
            elif np.any(jon.startswith(oznaka) for oznaka in ["a","b","c"]):
                prefiksi_u_spektru = spektar[lambda df: 
                    (df.m_z >= prefiksna_masa + pomeraj - 0.5) &
                    (df.m_z <= prefiksna_masa + pomeraj + 0.5)                        
                ]

                broj_mogucih[jon] += 1 if (((prefiksna_masa + pomeraj) >= 91) and ((prefiksna_masa + pomeraj) <= 2000)) else 0

                if not len(prefiksi_u_spektru):
                    missing_ion_probs[jon] = missing_ion_probs.get(jon, 0) + (1 if ((prefiksna_masa + pomeraj) >= 91) and ((prefiksna_masa + pomeraj) <= 2000) else 0)


100%|█████████████████████████████████████| 500/500 [00:58<00:00,  8.48it/s]


In [11]:
for key in missing_ion_probs.keys():
    missing_ion_probs[key] = missing_ion_probs[key]/broj_mogucih[key]
    missing_ion_scores[key] = math.log(missing_ion_probs[key])

In [26]:
skor_po_rangovima = {
rang: {
        jon: math.log(eksperimentalna_verovatnoca_jona_po_rangovima[rang][jon]/(eksperimentalna_verovatnoca_jona_po_rangovima[rang]["noise"]/100)) \
        if (eksperimentalna_verovatnoca_jona_po_rangovima[rang][jon] != 0) and (eksperimentalna_verovatnoca_jona_po_rangovima[rang]["noise"] != 0) \
        else math.log(eksperimentalna_verovatnoca_jona_po_rangovima[rang][jon]/0.0001) if (eksperimentalna_verovatnoca_jona_po_rangovima[rang][jon] != 0) \
        else 0
            for jon in po_jonima if jon != "noise"
    } for rang, po_jonima in eksperimentalna_verovatnoca_jona_po_rangovima.to_dict().items()
}

skor_po_rangovima_po_jonima = pd.DataFrame.from_dict(skor_po_rangovima, orient='index')

In [27]:
skor_po_rangovima_po_jonima.loc["-1"] = missing_ion_scores

In [28]:
skor_po_rangovima_po_jonima

,y,b,b-h2o,a,b-nh3,y-nh3,b-h2o-h2o
1,7.412475,5.596810,4.605170,1.469676,3.415586,1.469676,2.162823
2,7.459548,5.574571,4.772224,1.514128,3.123566,2.612740,2.612740
3,7.991979,5.679685,5.010635,3.064725,3.575551,0.000000,2.659260
4,7.018530,4.763394,4.307919,1.742969,2.148434,0.000000,1.742969
5,7.300317,5.354407,4.714369,0.000000,2.445686,0.000000,2.040221
6,7.673223,5.543440,5.270146,3.506558,2.813411,0.000000,3.324236
7,7.370231,5.214936,4.430817,2.079442,3.178054,0.000000,2.484907
8,6.812921,4.738702,3.811940,1.560648,2.253795,0.867501,1.560648
9,7.427631,5.733635,4.772224,2.612740,2.207275,2.900422,3.123566
10,5.912684,5.320790,3.711352,0.415515,1.514128,3.551010,0.415515


In [30]:
skor_po_rangovima_po_jonima.to_csv("izracunavanje-parametara-skoringa/parametri/skor_po_rangovima_po_jonima.csv")